# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with "mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray"

In [4]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'fruits_db', 'local', 'uk_food']


In [7]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [8]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [9]:
# review the collections in our new database
collections = db.list_collection_names()
print("Collections in 'uk_food':", collections)

Collections in 'uk_food': ['establishments']


In [10]:
# review a document in the establishments collection
establishments = db["establishments"]
from pprint import pprint
pprint(establishments.find_one())

{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('675892b2368144688544b6c3'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [12]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [13]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)

In [14]:
# Check that the new restaurant was inserted
print(f"New restaurant inserted with ID: {result.inserted_id}")

New restaurant inserted with ID: 675894a14d692e43b2ccef80


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [15]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
result = establishments.find_one(query, projection)
print("BusinessTypeID for 'Restaurant/Cafe/Canteen':")
pprint(result)

BusinessTypeID for 'Restaurant/Cafe/Canteen':
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [16]:
# Update the new restaurant with the correct BusinessTypeID
filter_query = {"BusinessName": "Penang Flavours"}
update_operation = {"$set": {"BusinessTypeID": "1"}}
update_result = establishments.update_one(filter_query, update_operation)


In [17]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("The new restaurant has been updated with the correct BusinessTypeID.")
else:
    print("No document was updated. Ensure the restaurant exists in the database.")

The new restaurant has been updated with the correct BusinessTypeID.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [18]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
dover_count = establishments.count_documents(query)
print(f"Number of establishments with LocalAuthorityName as 'Dover': {dover_count}")

Number of establishments with LocalAuthorityName as 'Dover': 994


In [19]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}
delete_result = establishments.delete_many(query)
print(f"Number of documents deleted: {delete_result.deleted_count}")

Number of documents deleted: 994


In [20]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}
remaining_count = establishments.count_documents(query)

In [21]:
# Check that other documents remain with 'find_one'
non_dover_document = establishments.find_one({"LocalAuthorityName": {"$ne": "Dover"}})
if non_dover_document:
    print("A document that is not from 'Dover' exists:")
    from pprint import pprint
    pprint(non_dover_document)
else:
    print("No documents remain in the collection.")

A document that is not from 'Dover' exists:
{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('675892b2368144688544b9aa'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [22]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},  
    [
        {
            "$set": {
                "geocode.latitude": {"$toDouble": "$geocode.latitude"},
                "geocode.longitude": {"$toDouble": "$geocode.longitude"}
            }
        }
    ]
)

print("Conversion of geocode.latitude and geocode.longitude from string to decimal completed.")

Conversion of geocode.latitude and geocode.longitude from string to decimal completed.


Use `update_many` to convert `RatingValue` to integer numbers.

In [23]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [24]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string"}},  
    [
        {
            "$set": {
                "RatingValue": {"$toInt": "$RatingValue"} 
            }
        }
    ]
)

print("RatingValue has been successfully converted from string to integer.")

RatingValue has been successfully converted from string to integer.


In [25]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one({"RatingValue": {"$ne": None}}, {"RatingValue": 1, "_id": 0})

print("Sample document after conversion:")
pprint(sample_document)

# Check the data type
if sample_document:
    print(f"\nData type of RatingValue: {type(sample_document['RatingValue'])}")

Sample document after conversion:
{'RatingValue': 4}

Data type of RatingValue: <class 'int'>
